In [3]:
import os
import numpy as np
import cv2
from PIL import Image

# 이미지 경로
dir_path = '/home/mmc/disk2/duck/cap/U-2-Net-master/test_data/images'
mix_path = '/home/mmc/disk2/duck/cap/U-2-Net-master/test_data/u2net_results'

# 저장 경로
save_path = '/home/mmc/disk2/duck/cap/U-2-Net-master/test_data/mix_val_data'


# 이미지 파일 리스트
dir_files = sorted([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
mix_files = sorted([f for f in os.listdir(mix_path) if os.path.isfile(os.path.join(mix_path, f))])

# 이미지 처리
for dir_file, mix_file in zip(dir_files, mix_files):
    # 이미지 열기
    dir_image = Image.open(os.path.join(dir_path, dir_file))
    mix_image = Image.open(os.path.join(mix_path, mix_file))

    # 이미지를 numpy 배열로 변환
    dir_array = np.array(dir_image)
    mix_array = np.array(mix_image)

    # 배경 이미지 생성 (invert_mix_array)
    background = cv2.bitwise_not(mix_array)

    # u2net 거친 후 흰색 마스크 생성
    white_mask = (mix_array[:, :, 0] >= 150) & (mix_array[:, :, 1] >= 150) & (mix_array[:, :, 2] >= 150)
    white_mask_3d = np.stack([white_mask]*3, axis=-1)

    # 최종 결과 이미지 생성
    result_array = np.where(white_mask_3d, dir_array, background)
    result_array = np.clip(result_array, 0, 255)  # 값 조정

    # 결과 저장
    result_image = Image.fromarray(result_array.astype(np.uint8))
    result_image.save(os.path.join(save_path, dir_file))

print("처리 완료!")


처리 완료!
